In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import sys
from pathlib import Path
from dotenv import load_dotenv

project_root = Path().cwd().parent
sys.path.append(str(project_root))
load_dotenv(project_root / "secrets/.env")

from fdua_competition.vectorstore import build_vectorstore
from fdua_competition.enums import Mode, VectorStoreOption
from fdua_competition.rag import ResearchAssistant
from pprint import pprint

In [32]:
vectorstore = build_vectorstore("output_simple_test", Mode.TEST, VectorStoreOption.CHROMA)

[prepare_vectorstore] chroma: /fdua-competition/.fdua-competition/vectorstores/chroma
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/9.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/8.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/10.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/6.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/7.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/5.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-competition/.fdua-competition/validation/documents/4.pdf
[add_document_to_vectorstore] skipping existing document: /fdua-co

In [33]:
assistant = ResearchAssistant(vectorstore)

In [34]:
res = assistant.invoke("電通についてまとめて")

In [35]:
pprint(res)

ResearchAssistantResponse(query='電通についてまとめて', response='電通は日本の広告代理店で1901年に創立され広告業界の発展に寄与するユニークなビジネスモデルを持つ', reason='電通の創立や広告業界での役割に関する情報を要約した。', organization_name='電通グループ', contexts=['年に創設された日本広告1901光永星郎の手によって・創業者', '広告専業の会社として再スタート、国策により通信部門を手放し'])
